## Installing necessary libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 300)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pointbiserialr
import pingouin as pg
import matplotlib.pyplot as plt

/home/jupyter-apaddo/.local/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/home/jupyter-apaddo/.local/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Loading the merged dataframe

In [2]:
woundassessment=pd.read_csv("Final_Merged_DataFrame_June24.csv")
woundassessment=woundassessment.drop(["Unnamed: 0"],axis=1)

#WoundLocation are stored in Wound_Title column
woundassessment=woundassessment.rename(columns={"Wound_Title":"WoundLocation"})
woundassessment.columns.tolist()

#Dropping some columns which are not relevant for classification
woundassessment=woundassessment.drop(["WoundID", "PatientID", "WoundNumber"],axis=1)

#PatientIDWoundNumber is showing a floating point number having a 0 added to it, changing it.
woundassessment['PatientIDWoundNumber'] = woundassessment['PatientIDWoundNumber'].astype(str).str.replace('\.0', '', regex=True)

#Dropping duplicates
woundassessment=woundassessment.drop_duplicates()

woundassessment=woundassessment.reset_index()
woundassessment=woundassessment.drop(["index"],axis=1)

/tmp/ipykernel_220436/1687845697.py:1: DtypeWarning: Columns (61,62,63,64,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,141,147,154) have mixed types. Specify dtype option on import or set low_memory=False.
  woundassessment=pd.read_csv("Final_Merged_DataFrame_June24.csv")


### Check the Wound Location

In [3]:
woundassessment["WoundLocation"]

0             Wound #1 Right, Plantar Foot
1             Wound #1 Right, Plantar Foot
2             Wound #1 Right, Plantar Foot
3             Wound #1 Right, Plantar Foot
4             Wound #1 Right, Plantar Foot
                        ...               
309061       Wound #2 Right, Plantar Foot 
309062       Wound #2 Right, Plantar Foot 
309063    Wound #3 Right, Posterior Ankle 
309064    Wound #3 Right, Posterior Ankle 
309065    Wound #3 Right, Posterior Ankle 
Name: WoundLocation, Length: 309066, dtype: object

### We don't need left-right information here, thus removing that

In [4]:
# Correcting the Wound Location values
woundassessment['WoundLocation'] = woundassessment['WoundLocation'].str.split(', ').str[1].str.strip()
woundassessment["WoundLocation"]

0            Plantar Foot
1            Plantar Foot
2            Plantar Foot
3            Plantar Foot
4            Plantar Foot
               ...       
309061       Plantar Foot
309062       Plantar Foot
309063    Posterior Ankle
309064    Posterior Ankle
309065    Posterior Ankle
Name: WoundLocation, Length: 309066, dtype: object

## Dropping the 'ID' columns because we do not need those for classification and few columns which are not relevant to predict wound healing status

In [5]:
woundassessment=woundassessment.drop(["Woundassessment_UserID","patientmedications_DocumentType","patientmedications_VisitID",
                                      "patientmedications_UserID","Wound_DocumentType","Wound_UserID","Wound_PhysicianID", 
                                      "AdmissionID", "previousID", "SyncUserID","ReferringPhysicianID", "patientmedications_Title", 
                                      "patientmedications_PhysicianID", "Woundassessment_PhysicianID","Patient_UserID","PhysicianID",
                                      "Woundassessment_VisitID", "Notes", "imageFileCount", "Woundassessment_Title", 
                                      "Woundassessment_DocumentType"], axis=1)
woundassessment

,Pulse,AdherentYellowSlough,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,...,SmokingStatusText,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,TInsuranceZip,PInsuranceZip,Diabetes
0,NaN,NaN,1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,-1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309061,NaN,NaN,1.0,1.0,93.7,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309062,NaN,NaN,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309063,NaN,NaN,1.0,1.0,81.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309064,NaN,NaN,1.0,1.0,67.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Check which columns have how many non-null values

In [6]:
# Get the number of non-null values in each column
non_null_counts = woundassessment.count()

# Get the number of null values in each column
null_counts = woundassessment.isnull().sum()

# Output the counts
print("Non-null counts:")
#print(non_null_counts)
#print("\nNull counts:")
#print(null_counts)

non_null_counts

Non-null counts:


Pulse                                     71072
AdherentYellowSlough                       2176
AdiposeExposed                           303691
AdiposeNecrosisExposed                   303662
area                                     258097
AtrophieBlanche                          303692
BoneNecrosisExposed                      303662
BrawnyInduration                         303692
Callus                                   303692
CapsuleExposed                           303663
CapsuleNecrosisExposed                   303662
color                                    207881
CompCompression                          175054
CompDiet                                 284040
CompDressing                             284040
CompExercise                             284040
CompHBOVisits                            284039
CompLimb                                 284040
CompMedication                           284040
CompOffLoad                              284040
CompSmoking                             

## Drop those columns which have less than 247252 values (non-null values)

In [7]:
# remove those columns which have <20% values (thresh=247252)
# check the code book while combining similar features and refer to tbl_CODE_feature.csv (to find the feature info)

In [8]:
woundassessment = woundassessment.dropna(axis=1, thresh=247252)
woundassessment

,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,Religion,SecondaryRaceCodeList,DateAdded_patient,Patient_Title,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber
0,1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","['3/17/2017 5:39:51 PM', '3/17/2017 5:39:51 PM...","['Patient', 'Patient', 'Patient', 'Patient', '...","['PatientDocument', 'PatientDocument', 'Patien...",71.81,Malnourished,0.0,3.0,5978401_1
1,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","['3/17/2017 5:39:51 PM', '3/17/2017 5:39:51 PM...","['Patient', 'Patient', 'Patient', 'Patient', '...","['PatientDocument', 'PatientDocument', 'Patien...",71.81,Malnourished,0.0,3.0,5978401_1
2,-1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","['3/17/2017 5:39:51 PM', '3/17/2017 5:39:51 PM...","['Patient', 'Patient', 'Patient', 'Patient', '...","['PatientDocument', 'PatientDocument', 'Patien...",71.82,Malnourished,0.0,3.0,5978401_1
3,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","['3/17/2017 5:39:51 PM', '3/17/2017 5:39:51 PM...","['Patient', 'Patient', 'Patient', 'Patient', '...","['PatientDocument', 'PatientDocument', 'Patien...",71.82,Malnourished,0.0,3.0,5978401_1
4,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","['3/17/2017 5:39:51 PM', '3/17/2017 5:39:51 PM...","['Patient', 'Patient', 'Patient', 'Patient', '...","['PatientDocument', 'PatientDocument', 'Patien...",71.82,Malnourished,0.0,3.0,5978401_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309061,1.0,1.0,93.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[77.0, 77.0]","[nan, nan]","['1/11/2023 3:24:10 PM', '1/11/2023 3:24:10 PM']","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",59.26,NaN,NaN,NaN,10210112_2
309062,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[77.0, 77.0]","[nan, nan]","['1/11/2023 3:24:10 PM', '1/11/2023 3:24:10 PM']","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",59.26,NaN,NaN,NaN,10210112_2
309063,1.0,1.0,81.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[77.0, 77.0]","[nan, nan]","['1/11/2023 3:24:10 PM', '1/11/2023 3:24:10 PM']","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",59.26,NaN,NaN,NaN,10210112_3
309064,1.0,1.0,67.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"[77.0, 77.0]","[nan, nan]","['1/11/2023 3:24:10 PM', '1/11/2023 3:24:10 PM']","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",59.26,NaN,NaN,NaN,10210112_3


## See how many unique wounds are there for which we want to do the classification and sort the dataframe based on the WoundNumber

In [9]:
display(woundassessment["PatientIDWoundNumber"].nunique())
sorted_df = woundassessment.sort_values('PatientIDWoundNumber')

25255

## See how many unique patients are there for whom we want to do the classification

In [10]:
sorted_df["PatientIDWoundNumber"].str.split('_').str[0].nunique()

8728

#### We want to have a processed dataframe for the classification but in this dataframe there are a lot of data elements, so we want to keep only those data elements which are necessary/important for the classification models. Thus we need to do the feature importance, thus we are adding new column in this "wound healing status", using that as target variable, we will try to compute feature importance value for each data elements and based on that, we will remove the data elements before doing the transpose.

In [ ]:
big_df=pd.DataFrame()


sorted_df=sorted_df[sorted_df.DateAdded_woundassessment.notnull()]
#sorted_df["area"]=sorted_df["area"].replace(0,0.00001)

unique_wounds=sorted_df["PatientIDWoundNumber"].unique().tolist()
for i in range(len(unique_wounds)):
#     if i>500:
#         break
    print(i)
    
    wound=unique_wounds[i]
    df_wound=sorted_df.query("PatientIDWoundNumber==@wound")
    #df_wound=df_wound[["PatientIDWoundNumber","DateAdded_woundassessment","area"]]
    df_wound=df_wound.drop_duplicates()
    df_wound=df_wound[df_wound.area.notnull()]

    
    # Convert 'DateAdded_woundassessment' column to datetime type
    df_wound['DateAdded_woundassessment'] = pd.to_datetime(df_wound['DateAdded_woundassessment'])

    # Keep the maximum value in 'area' for each date
    df_wound = df_wound.loc[df_wound.groupby('DateAdded_woundassessment')['area'].idxmax()]

    df_wound=df_wound.sort_values('DateAdded_woundassessment')
    
    df_wound["Wound_healing_status"]=np.nan
    df_wound["Wound_healing_percentage"]=np.nan

    
    df_wound=df_wound.reset_index()
    df_wound=df_wound.drop(["index"],axis=1)
    
    for j in range(df_wound.shape[0]):
        if j == 0:
            df_wound["Wound_healing_status"][j]='Unknown'
        else:
            percentage=(df_wound["area"][j-1]-df_wound["area"][j])/df_wound["area"][j-1]*100
            df_wound["Wound_healing_percentage"][j]= percentage

            if percentage == 100:
                df_wound["Wound_healing_status"][j]= 'Healed'
            elif percentage>=65:
                df_wound["Wound_healing_status"][j]= 'Healing'
            elif (percentage<65) & (percentage>=20):
                df_wound["Wound_healing_status"][j]= 'Grey'
            else:
                df_wound["Wound_healing_status"][j]= 'Not healing'
    
    # Convert the 'Date' column to datetime
    df_wound['DateAdded_woundassessment'] = pd.to_datetime(df_wound['DateAdded_woundassessment'])

    # Find the minimum date in the DataFrame
    first_date = df_wound['DateAdded_woundassessment'].min()

    # Calculate the day differences relative to the first date
    df_wound['DayNumber'] = (df_wound['DateAdded_woundassessment'] - first_date).dt.days + 1
    
    
    big_df = pd.concat([big_df, df_wound], axis=0)

    #display(df_wound)
#sorted_df[["PatientIDWoundNumber","DateAdded_woundassessment","area"]].head(50)

In [12]:
big_df["Wound_healing_status"].unique()

array(['Unknown', 'Not healing', 'Grey', 'Healing', 'Healed'],
      dtype=object)

## We do not need the wound healing percentage column, thus we will remove that.

In [13]:
big_df=big_df.drop(["Wound_healing_percentage"], axis=1)

In [14]:
big_df.to_csv("big_df_preprocessed1.csv")

In [74]:
big_df=pd.read_csv("big_df_preprocessed1.csv")
big_df=big_df.drop(["Unnamed: 0"], axis=1)
big_df

/tmp/ipykernel_220436/1137423253.py:1: DtypeWarning: Columns (49,50,51,52,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  big_df=pd.read_csv("big_df_preprocessed1.csv")


,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,DateAdded_patient,Patient_Title,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber,Wound_healing_status,DayNumber
0,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['12/15/2022 8:44:18 AM', '12/15/2022 8:44:18 ...","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",65.73,At risk of malnutrition,11.0,3.0,10000074_1,Unknown,1
1,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['12/15/2022 8:44:18 AM', '12/15/2022 8:44:18 ...","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",65.75,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,8
2,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['12/15/2022 8:44:18 AM', '12/15/2022 8:44:18 ...","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",65.77,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,15
3,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['12/15/2022 8:44:18 AM', '12/15/2022 8:44:18 ...","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",65.79,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,22
4,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['12/15/2022 8:44:18 AM', '12/15/2022 8:44:18 ...","['Patient', 'Patient']","['PatientDocument', 'PatientDocument']",65.81,At risk of malnutrition,11.0,3.0,10000074_1,Grey,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['1/11/2023 8:54:50 AM', '1/11/2023 8:54:50 AM...","['Patient', 'Patient', 'Patient']","['PatientDocument', 'PatientDocument', 'Patien...",68.86,Normal nutritional status,12.0,4.0,9996014_1,Not healing,72
197471,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['1/11/2023 8:54:50 AM', '1/11/2023 8:54:50 AM...","['Patient', 'Patient', 'Patient']","['PatientDocument', 'PatientDocument', 'Patien...",68.87,Normal nutritional status,12.0,4.0,9996014_1,Grey,75
197472,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['1/11/2023 8:54:50 AM', '1/11/2023 8:54:50 AM...","['Patient', 'Patient', 'Patient']","['PatientDocument', 'PatientDocument', 'Patien...",68.88,Normal nutritional status,12.0,4.0,9996014_1,Not healing,80
197473,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,"['1/11/2023 8:54:50 AM', '1/11/2023 8:54:50 AM...","['Patient', 'Patient', 'Patient']","['PatientDocument', 'PatientDocument', 'Patien...",68.90,Normal nutritional status,12.0,4.0,9996014_1,Not healing,87


### Changing the formats of data values in some columns which are in string format, and inside of the string they are stored in list formats

In [75]:
big_df["NonWoundDiagnosis"] = big_df['NonWoundDiagnosis'].str.extract(r'\[([^\]]+)\]')  # Extracts the value within brackets
big_df["NonWoundDiagnosis"]=big_df["NonWoundDiagnosis"].str.split(',').apply(lambda x: pd.Series(x).mode()[0])
big_df["NonWoundDiagnosis"]=big_df["NonWoundDiagnosis"].map({'False': False, ' False': False, ' True': True, ' nan': np.nan, 'nan': np.nan})

big_df["PalliativeCare"] = big_df['PalliativeCare'].str.extract(r'\[([^\]]+)\]')  # Extracts the value within brackets
big_df["PalliativeCare"]=big_df["PalliativeCare"].str.split(',').apply(lambda x: pd.Series(x).mode()[0])
big_df["PalliativeCare"]=big_df["PalliativeCare"].map({'False': False, ' False': False, ' True': True, ' nan': np.nan, 'nan': np.nan})
big_df["PalliativeCare"].unique()

big_df["CauseOfDeath"] = big_df['CauseOfDeath'].str.extract(r'\[([^\]]+)\]')  # Extracts the value within brackets
big_df["CauseOfDeath"]=big_df["CauseOfDeath"].str.split(',').apply(lambda x: pd.Series(x).mode()[0])
big_df["CauseOfDeath"]=big_df["CauseOfDeath"].replace(' nan', 'nan').replace(" 'Unk'", " 'unknown'")
big_df["CauseOfDeath"] = big_df["CauseOfDeath"].str.strip().str.replace("'", '')
big_df["CauseOfDeath"].unique()

array(['nan', 'Heart Failure', 'CVA', 'Cancer', 'unknown',
       'multisystem organ failure', 'Cardiopulm arrest',
       'Sepsis due to ulcerative colitis',
       'cardiopulmonary and neurologic functions had ceased',
       'Congestive Heart Failure', 'Cardiac arrest', 'COVID-19',
       'Stage 4 lung cancer', 'Respiratory related',
       'recurrent tonsillar squamous cell carcinoma with metastasis to the lungs and failure to thrive',
       'Cranial injuries due to fall', 'Septic shock'], dtype=object)

In [76]:
list_features_to_extract_from_string_list=["DateOfDeath", "ethnicityCdcCode", "patientEthnicity", "PatientGender", 
                                           "PatientSelfCare", "PatientZIP", "RaceCodeList", "Religion", 
                                           "SecondaryRaceCodeList", "DateAdded_patient", "MedicareAdmission", 
                                           "AdmissionDate", "AdmissionStatusCode", "Admission_dateAdded", 
                                           "StartDate", "Patient_DocumentType", "Patient_Title"]

for i in range(len(list_features_to_extract_from_string_list)):
    column_name=list_features_to_extract_from_string_list[i]
    big_df[column_name] = big_df[column_name].str.extract(r'\[([^\]]+)\]')  # Extracts the value within brackets
    big_df[column_name]=big_df[column_name].str.split(',').apply(lambda x: pd.Series(x).mode()[0])
    big_df[column_name]=big_df[column_name].replace(' nan', 'nan')
    big_df[column_name] = big_df[column_name].str.strip().str.replace("'", '')
    display(big_df[column_name].unique())

array(['nan', '12/3/2022 12:00:00 AM', '8/11/2017 12:00:00 AM',
       '9/30/2017 12:00:00 AM', '12/1/2016 12:00:00 AM',
       '6/13/2018 12:00:00 AM', '10/27/2022 12:00:00 AM',
       '12/23/2016 12:00:00 AM', '5/26/2018 12:00:00 AM',
       '1/17/2020 12:00:00 AM', '8/24/2016 12:00:00 AM',
       '8/28/2017 12:00:00 AM', '2/4/2020 12:00:00 AM',
       '6/30/2017 12:00:00 AM', '6/2/2021 12:00:00 AM',
       '9/6/2016 12:00:00 AM', '10/26/2020 12:00:00 AM',
       '4/21/2020 12:00:00 AM', '5/1/2021 12:00:00 AM',
       '9/18/2017 12:00:00 AM', '3/5/2019 12:00:00 AM',
       '3/8/2020 12:00:00 AM', '11/25/2021 12:00:00 AM',
       '4/11/2018 12:00:00 AM', '5/10/2017 12:00:00 AM',
       '1/16/2022 12:00:00 AM', '12/15/2017 12:00:00 AM',
       '9/14/2017 12:00:00 AM', '4/22/2020 12:00:00 AM',
       '12/12/2019 12:00:00 AM', '8/31/2021 12:00:00 AM',
       '10/8/2018 12:00:00 AM', '6/7/2018 12:00:00 AM',
       '4/10/2017 12:00:00 AM', '5/27/2019 12:00:00 AM',
       '3/8/2018 12:00:00

array(['2186-5', 'nan', '2135-2', 'asku'], dtype=object)

array(['9.0', 'nan', '10.0', '11.0'], dtype=object)

array(['F', 'M', 'nan'], dtype=object)

array(['nan', '1.0', '0.0', '-1.0'], dtype=object)

...

array(['2.0', 'nan'], dtype=object)

array(['10/3/2022 10:57', '10/3/2022 11:29', '10/3/2022 11:47', ...,
       '9/30/2022 11:51', '9/30/2022 12:02', '10/5/2022 11:31'],
      dtype=object)

array(['nan', '6/24/2015 9:21:21 AM', '4/4/2016 10:39:11 AM',
       '3/10/2017 2:49:01 PM', '3/10/2017 10:56:34 AM',
       '3/7/2017 2:46:26 PM', '3/7/2017 5:42:25 PM',
       '3/7/2017 2:43:18 PM', '3/7/2017 2:40:38 PM',
       '3/7/2017 3:39:16 PM', '3/7/2017 2:55:46 PM',
       '3/7/2017 2:53:29 PM', '3/7/2017 2:48:10 PM',
       '3/7/2017 2:55:40 PM', '3/13/2017 4:46:38 PM',
       '5/31/2021 10:13:43 AM'], dtype=object)

array(['PatientDocument', 'nan'], dtype=object)

array(['Patient', 'nan'], dtype=object)

In [77]:
big_df["MedicareAdmission"] = pd.to_numeric(big_df["MedicareAdmission"].replace('nan', np.nan), errors='coerce')

## Modifying some data values in some features according to the Codebook

#### Religion: 78 is Unknown, so changing all NaN to 78

In [78]:
big_df["Religion"].value_counts()

Religion
77.0    114591
79.0     18343
16.0     14826
13.0     14419
57.0      7972
39.0      7837
78.0      3597
54.0      2891
56.0      2002
9.0       1901
37.0      1853
nan       1830
31.0       850
46.0       719
22.0       612
76.0       529
19.0       507
23.0       454
81.0       424
32.0       377
35.0       166
12.0       157
15.0       151
11.0        99
27.0        83
71.0        82
1.0         73
73.0        58
29.0        27
25.0        23
14.0         9
18.0         5
5.0          4
40.0         3
3.0          1
Name: count, dtype: int64

In [79]:
# Assuming your DataFrame is named 'df'
big_df['Religion'] = big_df['Religion'].replace('nan', '78')
big_df["Religion"].value_counts()

Religion
77.0    114591
79.0     18343
16.0     14826
13.0     14419
57.0      7972
39.0      7837
78.0      3597
54.0      2891
56.0      2002
9.0       1901
37.0      1853
78        1830
31.0       850
46.0       719
22.0       612
76.0       529
19.0       507
23.0       454
81.0       424
32.0       377
35.0       166
12.0       157
15.0       151
11.0        99
27.0        83
71.0        82
1.0         73
73.0        58
29.0        27
25.0        23
14.0         9
18.0         5
5.0          4
40.0         3
3.0          1
Name: count, dtype: int64

#### Stage

In [80]:
big_df["Stage"].value_counts(dropna=False)

Stage
14.0     41353
8.0      27029
10.0     19047
27.0     13969
NaN      10453
23.0     10387
6.0       9379
3.0       7954
34.0      6339
4.0       5954
15.0      4082
2.0       3430
35.0      3361
9.0       3167
7.0       3152
127.0     2432
39.0      1908
31.0      1890
41.0      1828
29.0      1442
124.0     1315
12.0      1116
26.0      1110
74.0      1061
13.0      1050
22.0       908
46.0       900
19.0       767
42.0       753
82.0       712
30.0       677
73.0       649
40.0       568
137.0      559
18.0       548
65.0       516
28.0       511
33.0       488
5.0        468
25.0       457
76.0       454
11.0       437
1.0        435
80.0       373
36.0       340
38.0       292
83.0       152
86.0       149
84.0       138
126.0       97
21.0        88
24.0        83
147.0       64
45.0        60
149.0       56
128.0       55
78.0        48
158.0       45
75.0        44
32.0        43
17.0        39
125.0       38
139.0       29
47.0        27
161.0       27
20.0        23
123.

#### We do not need such granularity in Stage feature so we want to see the Code book and make a top level stage for all the wounds during the visits.

In [81]:
## Firstly we are changing Null to 47 because according to codebook 47 means unknown or Null

In [82]:
big_df['Stage'] = big_df['Stage'].replace(np.nan, 47)
big_df["Stage"].value_counts(dropna=False)

Stage
14.0     41353
8.0      27029
10.0     19047
27.0     13969
47.0     10480
23.0     10387
6.0       9379
3.0       7954
34.0      6339
4.0       5954
15.0      4082
2.0       3430
35.0      3361
9.0       3167
7.0       3152
127.0     2432
39.0      1908
31.0      1890
41.0      1828
29.0      1442
124.0     1315
12.0      1116
26.0      1110
74.0      1061
13.0      1050
22.0       908
46.0       900
19.0       767
42.0       753
82.0       712
30.0       677
73.0       649
40.0       568
137.0      559
18.0       548
65.0       516
28.0       511
33.0       488
5.0        468
25.0       457
76.0       454
11.0       437
1.0        435
80.0       373
36.0       340
38.0       292
83.0       152
86.0       149
84.0       138
126.0       97
21.0        88
24.0        83
147.0       64
45.0        60
149.0       56
128.0       55
78.0        48
158.0       45
75.0        44
32.0        43
17.0        39
125.0       38
139.0       29
161.0       27
20.0        23
123.0       22
148.

In [83]:
df["WoundType"]

0          1
1          1
2          1
3          1
4          2
        ... 
1810     685
1811     685
1812    <NA>
1813    <NA>
1814    <NA>
Name: WoundType, Length: 1815, dtype: Int64

In [84]:
df = pd.read_csv("Stage_new_label.csv")
df=df.drop(["Unnamed: 0"], axis=1)
df["WoundType"] = pd.to_numeric(df["WoundType"], errors='coerce')
df["WoundType"] = df["WoundType"].astype('Int64')  # 'Int64' allows for NaN values in an integer column
df['Code_WoundType'] = df['Code'].astype(str) + '_' + df['WoundType'].astype(str)
df

,Code,WoundType,CustomSortOrder,CodeValue,New_label_stage,Code_WoundType
0,1,1,1.0,Stage 1 Pressure Injury,Stage 1 Pressure Injury,1_1
1,2,1,2.0,Stage 2 Pressure Injury,Stage 2 Pressure Injury,2_1
2,3,1,3.0,Stage 3 Pressure Injury,Stage 3 Pressure Injury,3_1
3,4,1,4.0,Stage 4 Pressure Injury,Stage 4 Pressure Injury,4_1
4,5,2,NaN,Grade 0,Stage 2 Pressure Injury,5_2
...,...,...,...,...,...,...
1810,185,685,NaN,Partial Thickness,Partial Thickness,185_685
1811,186,685,NaN,Unclassifiable,Unclassifiable,186_685
1812,187,<NA>,137.0,STAGE,STAGE,187_<NA>
1813,188,<NA>,138.0,Kennedy Terminal Ulcer,Kennedy Terminal Ulcer,188_<NA>


In [85]:
big_df['Stage']=big_df['Stage'].astype(int)

In [86]:
big_df['Stage_Type'] = big_df['Stage'].astype(str) + '_' + big_df['Type'].astype(str)
big_df

,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,Patient_Title,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber,Wound_healing_status,DayNumber,Stage_Type
0,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,65.73,At risk of malnutrition,11.0,3.0,10000074_1,Unknown,1,14_6
1,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,65.75,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,8,14_6
2,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,65.77,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,15,14_6
3,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,65.79,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,22,14_6
4,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,65.81,At risk of malnutrition,11.0,3.0,10000074_1,Grey,29,14_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,68.86,Normal nutritional status,12.0,4.0,9996014_1,Not healing,72,8_3
197471,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,68.87,Normal nutritional status,12.0,4.0,9996014_1,Grey,75,8_3
197472,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,68.88,Normal nutritional status,12.0,4.0,9996014_1,Not healing,80,8_3
197473,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Patient,PatientDocument,68.90,Normal nutritional status,12.0,4.0,9996014_1,Not healing,87,8_3


In [87]:
filter_stage_code_values = big_df["Stage_Type"].unique().tolist()

filtered_df = df[df['Code_WoundType'].isin(filter_stage_code_values)]
filtered_df["WoundType"]=filtered_df["WoundType"].astype('int')
filtered_df

/tmp/ipykernel_220436/1779767609.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["WoundType"]=filtered_df["WoundType"].astype('int')


,Code,WoundType,CustomSortOrder,CodeValue,New_label_stage,Code_WoundType
0,1,1,1.0,Stage 1 Pressure Injury,Stage 1 Pressure Injury,1_1
1,2,1,2.0,Stage 2 Pressure Injury,Stage 2 Pressure Injury,2_1
2,3,1,3.0,Stage 3 Pressure Injury,Stage 3 Pressure Injury,3_1
3,4,1,4.0,Stage 4 Pressure Injury,Stage 4 Pressure Injury,4_1
4,5,2,NaN,Grade 0,Stage 2 Pressure Injury,5_2
5,6,2,NaN,Grade 1,Stage 2 Pressure Injury,6_2
6,7,3,NaN,Partial Thickness,Stage 3 Pressure Injury,7_3
7,8,3,NaN,Full Thickness,Stage 3 Pressure Injury,8_3
8,9,4,NaN,Partial Thickness,Stage 4 Pressure Injury,9_4
9,10,4,NaN,Full Thickness,Stage 4 Pressure Injury,10_4


In [88]:
## Make a dictionary from Code_WoundType to New_label_stage

In [89]:
document_dict_1 = pd.Series(filtered_df.New_label_stage.values,index = filtered_df.Code_WoundType).to_dict()
big_df['Updated_Stage'] = big_df['Stage_Type'].map(document_dict_1)
big_df

,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber,Wound_healing_status,DayNumber,Stage_Type,Updated_Stage
0,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.73,At risk of malnutrition,11.0,3.0,10000074_1,Unknown,1,14_6,Stage 2 Pressure Injury
1,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.75,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,8,14_6,Stage 2 Pressure Injury
2,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.77,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,15,14_6,Stage 2 Pressure Injury
3,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.79,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,22,14_6,Stage 2 Pressure Injury
4,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.81,At risk of malnutrition,11.0,3.0,10000074_1,Grey,29,14_6,Stage 2 Pressure Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.86,Normal nutritional status,12.0,4.0,9996014_1,Not healing,72,8_3,Stage 3 Pressure Injury
197471,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.87,Normal nutritional status,12.0,4.0,9996014_1,Grey,75,8_3,Stage 3 Pressure Injury
197472,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.88,Normal nutritional status,12.0,4.0,9996014_1,Not healing,80,8_3,Stage 3 Pressure Injury
197473,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.90,Normal nutritional status,12.0,4.0,9996014_1,Not healing,87,8_3,Stage 3 Pressure Injury


In [90]:
big_df=big_df.drop(["Stage", "Type"], axis=1)
big_df

,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber,Wound_healing_status,DayNumber,Stage_Type,Updated_Stage
0,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.73,At risk of malnutrition,11.0,3.0,10000074_1,Unknown,1,14_6,Stage 2 Pressure Injury
1,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.75,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,8,14_6,Stage 2 Pressure Injury
2,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.77,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,15,14_6,Stage 2 Pressure Injury
3,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.79,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,22,14_6,Stage 2 Pressure Injury
4,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.81,At risk of malnutrition,11.0,3.0,10000074_1,Grey,29,14_6,Stage 2 Pressure Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.86,Normal nutritional status,12.0,4.0,9996014_1,Not healing,72,8_3,Stage 3 Pressure Injury
197471,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.87,Normal nutritional status,12.0,4.0,9996014_1,Grey,75,8_3,Stage 3 Pressure Injury
197472,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.88,Normal nutritional status,12.0,4.0,9996014_1,Not healing,80,8_3,Stage 3 Pressure Injury
197473,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.90,Normal nutritional status,12.0,4.0,9996014_1,Not healing,87,8_3,Stage 3 Pressure Injury


In [91]:
big_df["Updated_Stage"].value_counts(dropna=False)

Updated_Stage
Stage 2 Pressure Injury                                                     99829
Stage 3 Pressure Injury                                                     50134
Stage 4 Pressure Injury                                                     24114
NaN                                                                         15886
Unstageable Pressure Injury Obscured full-thickness skin and tissue loss     4630
Deep Tissue Pressure Injury Persistent non-blanchable deep red               1279
Unstageable/Unclassified                                                      559
Stage 1 Pressure Injury                                                       435
Full Thickness                                                                418
Eschar covered                                                                 88
Partial Thickness                                                              74
Medical Device Related Pressure Injury                                         29
Na

In [92]:
big_df.to_csv("big_df_preprocessed2.csv")

In [115]:
big_df=pd.read_csv("big_df_preprocessed2.csv")
big_df=big_df.drop(["Unnamed: 0"], axis=1)
big_df

/tmp/ipykernel_220436/3424413851.py:1: DtypeWarning: Columns (49,50,51,52,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,118,119,120,121,122,131) have mixed types. Specify dtype option on import or set low_memory=False.
  big_df=pd.read_csv("big_df_preprocessed2.csv")


,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber,Wound_healing_status,DayNumber,Stage_Type,Updated_Stage
0,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.73,At risk of malnutrition,11.0,3.0,10000074_1,Unknown,1,14_6,Stage 2 Pressure Injury
1,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.75,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,8,14_6,Stage 2 Pressure Injury
2,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.77,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,15,14_6,Stage 2 Pressure Injury
3,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.79,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,22,14_6,Stage 2 Pressure Injury
4,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.81,At risk of malnutrition,11.0,3.0,10000074_1,Grey,29,14_6,Stage 2 Pressure Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.86,Normal nutritional status,12.0,4.0,9996014_1,Not healing,72,8_3,Stage 3 Pressure Injury
197471,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.87,Normal nutritional status,12.0,4.0,9996014_1,Grey,75,8_3,Stage 3 Pressure Injury
197472,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.88,Normal nutritional status,12.0,4.0,9996014_1,Not healing,80,8_3,Stage 3 Pressure Injury
197473,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.90,Normal nutritional status,12.0,4.0,9996014_1,Not healing,87,8_3,Stage 3 Pressure Injury


In [116]:
## We have wrong value in "SloughLessThan20" column where the values is 2. We will remove that row.

In [117]:
big_df = big_df[big_df["SloughLessThan20"] != 2.0]
big_df

,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,CompDiet,...,Patient_DocumentType,Age,Patient_RiskLevel,Patient_Score,SmokingStatus,PatientIDWoundNumber,Wound_healing_status,DayNumber,Stage_Type,Updated_Stage
0,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.73,At risk of malnutrition,11.0,3.0,10000074_1,Unknown,1,14_6,Stage 2 Pressure Injury
1,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.75,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,8,14_6,Stage 2 Pressure Injury
2,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.77,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,15,14_6,Stage 2 Pressure Injury
3,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.79,At risk of malnutrition,11.0,3.0,10000074_1,Not healing,22,14_6,Stage 2 Pressure Injury
4,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,65.81,At risk of malnutrition,11.0,3.0,10000074_1,Grey,29,14_6,Stage 2 Pressure Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.86,Normal nutritional status,12.0,4.0,9996014_1,Not healing,72,8_3,Stage 3 Pressure Injury
197471,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.87,Normal nutritional status,12.0,4.0,9996014_1,Grey,75,8_3,Stage 3 Pressure Injury
197472,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.88,Normal nutritional status,12.0,4.0,9996014_1,Not healing,80,8_3,Stage 3 Pressure Injury
197473,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,PatientDocument,68.90,Normal nutritional status,12.0,4.0,9996014_1,Not healing,87,8_3,Stage 3 Pressure Injury


### SMALLINT=Number (Yes=-1, No=1, Unknown=0)
### TINYINT= Boolean (1=True/Yes, 0=False/No)
#### Converting those features according to this rule for SMALLINT

In [118]:
list_columns=['AdiposeExposed', 'AdiposeNecrosisExposed', 'AtrophieBlanche', 'BoneNecrosisExposed', 'BrawnyInduration', 
              'Callus', 'CapsuleExposed', 'CapsuleNecrosisExposed', 'Crepitus', 'Cyanosis', 'Denuded', 'DryScaly', 
              'Ecchymosis', 'Edema', 'Erythema', 'Excoriation', 'ExposedBone', 'ExposedJoint', 'ExposedMuscle', 
              'ExposedTendon', 'Fluctuance', 'Friable', 'greygran', 'Hemosiderosis', 'hypergran', 'Induration', 
              'JointNecrosisExposed', 'Maceration', 'Moist', 'MuscleNecrosisExposed', 'Pallor', 'PatientSelfCare', 
              'pinkgran', 'Rash', 'redgran', 'ResolvedRecurrence', 'ResultOfAccident', 'Rubor', 'Scarring', 
              'Shiny', 'sinustract', 'SystematicInfection', 'TendonNecrosisExposed', 'TunnelingOrUndermining',
              'MedicareAdmission', "SloughLessThan20"]
for i in range(len(list_columns)):
    column_name=list_columns[i]
    print("For Feature", column_name, ":")
    display(big_df[column_name].value_counts(dropna=False))
    big_df[column_name] = big_df[column_name].replace({0: np.nan, 1: False, -1: True})
    display(big_df[column_name].value_counts(dropna=False))
    # Convert the column to boolean
    big_df[column_name] = big_df[column_name].astype('boolean')
    display(big_df[column_name].value_counts(dropna=False))
    print("\n\n")

For Feature AdiposeExposed :


AdiposeExposed
 1.0    159607
 0.0     31819
-1.0      5628
 NaN       420
Name: count, dtype: int64

/tmp/ipykernel_220436/268784578.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df[column_name] = big_df[column_name].replace({0: np.nan, 1: False, -1: True})


AdiposeExposed
False    159607
NaN       32239
True       5628
Name: count, dtype: int64

/tmp/ipykernel_220436/268784578.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df[column_name] = big_df[column_name].astype('boolean')


...

SloughLessThan20
 1.0    99217
-1.0    48932
 0.0    46799
 NaN     2526
Name: count, dtype: int64

/tmp/ipykernel_220436/268784578.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df[column_name] = big_df[column_name].replace({0: np.nan, 1: False, -1: True})


SloughLessThan20
False    99217
NaN      49325
True     48932
Name: count, dtype: int64

/tmp/ipykernel_220436/268784578.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df[column_name] = big_df[column_name].astype('boolean')


SloughLessThan20
False    99217
<NA>     49325
True     48932
Name: count, dtype: Int64

In [119]:
big_df.dtypes

AdiposeExposed                           boolean
AdiposeNecrosisExposed                   boolean
area                                     float64
AtrophieBlanche                          boolean
BoneNecrosisExposed                      boolean
BrawnyInduration                         boolean
Callus                                   boolean
CapsuleExposed                           boolean
CapsuleNecrosisExposed                   boolean
CompDiet                                 float64
CompDressing                             float64
CompExercise                             float64
CompHBOVisits                            float64
CompLimb                                 float64
CompMedication                           float64
CompOffLoad                              float64
CompSmoking                              float64
CompWoundVisits                          float64
Crepitus                                 boolean
Cyanosis                                 boolean
Denuded             

## Save the dataframe

In [120]:
big_df.to_csv("big_df.csv")